In [ ]:
!pip install youtube-comment-downloader


In [ ]:

import numpy as np
import pandas as pd



In [ ]:
from itertools import islice
from youtube_comment_downloader import *
import pandas as pd

link = input("Youtube links: ")
downloader = YoutubeCommentDownloader()
comments = downloader.get_comments_from_url(link, sort_by=SORT_BY_POPULAR)
all_comments = []
for comment in islice(comments, 200):
     all_comments.append(comment)
df = pd.DataFrame(all_comments)
df.to_csv('comments.csv')

Youtube links: https://www.youtube.com/watch?v=YcYXyI4PMOs


In [ ]:
#Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

# Import functions for data preprocessing & data preparation
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from string import punctuation
import nltk
import re

In [ ]:
data = pd.read_csv('comments.csv')
data.columns


Index(['Unnamed: 0', 'cid', 'text', 'time', 'author', 'channel', 'votes',
       'replies', 'photo', 'heart', 'reply', 'time_parsed'],
      dtype='object')

In [ ]:
data1=data.drop(['Unnamed: 0','cid','time','votes','replies','photo','heart','reply','time_parsed'],axis=1)
data1

,text,author,channel
0,"Sorry Dagger, but I don't want to play with yo...",@Penaey,UCfCvArkcglruXhsq1k5ZBKw
1,The rat grew a fifth leg 👀,@someguy9683,UCTf-QM5K2nNybO71kxm6xBw
2,Theres going to be so many people invisble wai...,@tannerhamilton6025,UCZ3Qwbx0HZzipGq2LhcNLOw
3,Some people who got early access already figur...,@spinyslasher6586,UC_4Q6oSKW4rRPckKW-n8ugQ
4,“It’s gonna be hard playing with one hand” bro...,@DioBrando-DIO,UCVWIwVeImsetlTzDTK7uYEQ
...,...,...,...
95,Her and moon knight would be a good combo,@coreymulvaney3557,UCb1BctDc4AzQI-_Cwojwy7g
96,You're C skill can heal you too,@kinglansing5516,UC4sSeVGA-AWxJJd9tc4dMpg
97,with Strategists like this who needs Duelists?,@TomboTime,UCuDo_nHFCFMuToDiDyYu9TQ
98,So nobody is going to talk about “HELA-HARD”💀,@Moistcomforter,UCRph8EN8tm7GhLnpVu6kG1Q


In [ ]:
data1.columns=['Comment','author','channel']
data1

,Comment,author,channel
0,"Sorry Dagger, but I don't want to play with yo...",@Penaey,UCfCvArkcglruXhsq1k5ZBKw
1,The rat grew a fifth leg 👀,@someguy9683,UCTf-QM5K2nNybO71kxm6xBw
2,Theres going to be so many people invisble wai...,@tannerhamilton6025,UCZ3Qwbx0HZzipGq2LhcNLOw
3,Some people who got early access already figur...,@spinyslasher6586,UC_4Q6oSKW4rRPckKW-n8ugQ
4,“It’s gonna be hard playing with one hand” bro...,@DioBrando-DIO,UCVWIwVeImsetlTzDTK7uYEQ
...,...,...,...
95,Her and moon knight would be a good combo,@coreymulvaney3557,UCb1BctDc4AzQI-_Cwojwy7g
96,You're C skill can heal you too,@kinglansing5516,UC4sSeVGA-AWxJJd9tc4dMpg
97,with Strategists like this who needs Duelists?,@TomboTime,UCuDo_nHFCFMuToDiDyYu9TQ
98,So nobody is going to talk about “HELA-HARD”💀,@Moistcomforter,UCRph8EN8tm7GhLnpVu6kG1Q


In [ ]:
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
data1["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data1["Comment"]]
data1["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data1["Comment"]]
data1["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data1["Comment"]]
data1['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in data1["Comment"]]
score = data1["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append('Positive')
    elif i <= -0.05 :
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
data1["Sentiment"] = sentiment
data1.head()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Comment,author,channel,Positive,Negative,Neutral,Compound,Sentiment
0,"Sorry Dagger, but I don't want to play with yo...",@Penaey,UCfCvArkcglruXhsq1k5ZBKw,0.000,0.418,0.582,-0.4655,Negative
1,The rat grew a fifth leg 👀,@someguy9683,UCTf-QM5K2nNybO71kxm6xBw,0.000,0.000,1.000,0.0000,Neutral
2,Theres going to be so many people invisble wai...,@tannerhamilton6025,UCZ3Qwbx0HZzipGq2LhcNLOw,0.000,0.000,1.000,0.0000,Neutral
3,Some people who got early access already figur...,@spinyslasher6586,UC_4Q6oSKW4rRPckKW-n8ugQ,0.050,0.175,0.775,-0.8020,Negative
4,“It’s gonna be hard playing with one hand” bro...,@DioBrando-DIO,UCVWIwVeImsetlTzDTK7uYEQ,0.253,0.096,0.651,0.6808,Positive


In [ ]:
data2=data1.drop(['Positive','Negative','Neutral','Compound'],axis=1)
data2.head()

,Comment,author,channel,Sentiment
0,"Sorry Dagger, but I don't want to play with yo...",@Penaey,UCfCvArkcglruXhsq1k5ZBKw,Negative
1,The rat grew a fifth leg 👀,@someguy9683,UCTf-QM5K2nNybO71kxm6xBw,Neutral
2,Theres going to be so many people invisble wai...,@tannerhamilton6025,UCZ3Qwbx0HZzipGq2LhcNLOw,Neutral
3,Some people who got early access already figur...,@spinyslasher6586,UC_4Q6oSKW4rRPckKW-n8ugQ,Negative
4,“It’s gonna be hard playing with one hand” bro...,@DioBrando-DIO,UCVWIwVeImsetlTzDTK7uYEQ,Positive


In [ ]:
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
porter_stemmer = PorterStemmer()
lancaster_stemmer = LancasterStemmer()
snowball_stemer = SnowballStemmer(language="english")
lzr = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def text_processing(text):
    # convert text into lowercase
    text = text.lower()

    # remove new line characters in text
    text = re.sub(r'\n',' ', text)

    # remove punctuations from text
    text = re.sub('[%s]' % re.escape(punctuation), "", text)

    # remove references and hashtags from text
    text = re.sub("^a-zA-Z0-9$,.", "", text)

    # remove multiple spaces from text
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # remove special characters from text
    text = re.sub(r'\W', ' ', text)

    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])

    # stemming using porter stemmer from nltk package - msh a7sn 7aga - momken: lancaster, snowball
    # text=' '.join([porter_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([lancaster_stemmer.stem(word) for word in word_tokenize(text)])
    # text=' '.join([snowball_stemer.stem(word) for word in word_tokenize(text)])

    # lemmatizer using WordNetLemmatizer from nltk package
    text=' '.join([lzr.lemmatize(word) for word in word_tokenize(text)])

    return text

In [ ]:
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('omw-1.4')
data_copy = data2.copy()
data_copy.Comment = data_copy.Comment.apply(lambda text: text_processing(text))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [ ]:
processed_data = {
    'Sentence':data_copy.Comment,
    'Sentiment':data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()

,Sentence,Sentiment
0,sorry dagger dont want play anymore,0
1,rat grew fifth leg,1
2,there going many people invisble waiting near ...,1
3,people got early access already figured melee ...,0
4,gon na hard playing one hand bro always hard p...,2


In [ ]:
processed_data['Sentiment'].value_counts()


,count
Sentiment,
2,36
1,35
0,29


In [ ]:
df_neutral = processed_data[(processed_data['Sentiment']==1)]
df_negative = processed_data[(processed_data['Sentiment']==0)]
df_positive = processed_data[(processed_data['Sentiment']==2)]

# upsample minority classes
df_negative_upsampled = resample(df_negative,
                                 replace=True,
                                 n_samples= 205,
                                 random_state=42)

df_neutral_upsampled = resample(df_neutral,
                                 replace=True,
                                 n_samples= 205,
                                 random_state=42)


# Concatenate the upsampled dataframes with the neutral dataframe
final_data = pd.concat([df_negative_upsampled,df_neutral_upsampled,df_positive])

In [ ]:
final_data['Sentiment'].value_counts()

,count
Sentiment,
0,205
1,205
2,36


In [ ]:
corpus = []
for sentence in final_data['Sentence']:
    corpus.append(sentence)
corpus[0:5]

['punch enemy ult see doesnt matter use ability also take damage passive immediately become visible',
 'joelconolly5574 mean think people problem way theyre branding malice evil sue supposed allegory sa mental illness',
 'somewhat broken compared healer',
 'nah really first decendent 100 gooner bait game lost 90 player less month concord wasnt good game launching 60 buck price tag market every shooter f2p quite business choice',
 'renzotkac1236 stop talking']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = final_data.iloc[:, -1].values

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[56,  5,  0],
       [ 0, 62,  1],
       [ 4,  2,  4]])

In [ ]:
nb_score = accuracy_score(y_test, y_pred)
print('accuracy',nb_score)

accuracy 0.9104477611940298


In [ ]:
ok=pd.read_csv('comments.csv')
ok.head()

,Unnamed: 0,cid,text,time,author,channel,votes,replies,photo,heart,reply,time_parsed
0,0,UgyBvmGi2edsP8II9OR4AaABAg,"Sorry Dagger, but I don't want to play with yo...",1 day ago,@Penaey,UCfCvArkcglruXhsq1k5ZBKw,386,6.0,https://yt3.ggpht.com/PoKYOq1nyfmNwQBOwvJcpxRN...,False,False,1.736519e+09
1,1,Ugxc-H1zW6zflmPjiTF4AaABAg,The rat grew a fifth leg 👀,1 day ago,@someguy9683,UCTf-QM5K2nNybO71kxm6xBw,313,5.0,https://yt3.ggpht.com/ytc/AIdro_kNkXgzZ4SG2Kfa...,False,False,1.736519e+09
2,2,UgwgxW4UBh4HQLgqIYZ4AaABAg,Theres going to be so many people invisble wai...,1 day ago,@tannerhamilton6025,UCZ3Qwbx0HZzipGq2LhcNLOw,176,2.0,https://yt3.ggpht.com/aDgWySk6pz2fKwlEDBenFLrg...,False,False,1.736519e+09
3,3,UgxjC079wVdacJqnaol4AaABAg,Some people who got early access already figur...,1 day ago,@spinyslasher6586,UC_4Q6oSKW4rRPckKW-n8ugQ,80,3.0,https://yt3.ggpht.com/ytc/AIdro_mboSMdfByfd71Q...,False,False,1.736519e+09
4,4,UgwZVr4i_K-Pgy1VDoZ4AaABAg,“It’s gonna be hard playing with one hand” bro...,1 day ago,@DioBrando-DIO,UCVWIwVeImsetlTzDTK7uYEQ,56,1.0,https://yt3.ggpht.com/kODcOv2zmjMYSg4iReod1BGh...,False,False,1.736519e+09


In [ ]:
data=ok.drop(['Unnamed: 0','cid','time','votes','replies','photo','heart','reply','time_parsed'],axis=1)
data.columns=['comments','author','channel']
data

,comments,author,channel
0,"Sorry Dagger, but I don't want to play with yo...",@Penaey,UCfCvArkcglruXhsq1k5ZBKw
1,The rat grew a fifth leg 👀,@someguy9683,UCTf-QM5K2nNybO71kxm6xBw
2,Theres going to be so many people invisble wai...,@tannerhamilton6025,UCZ3Qwbx0HZzipGq2LhcNLOw
3,Some people who got early access already figur...,@spinyslasher6586,UC_4Q6oSKW4rRPckKW-n8ugQ
4,“It’s gonna be hard playing with one hand” bro...,@DioBrando-DIO,UCVWIwVeImsetlTzDTK7uYEQ
...,...,...,...
95,Her and moon knight would be a good combo,@coreymulvaney3557,UCb1BctDc4AzQI-_Cwojwy7g
96,You're C skill can heal you too,@kinglansing5516,UC4sSeVGA-AWxJJd9tc4dMpg
97,with Strategists like this who needs Duelists?,@TomboTime,UCuDo_nHFCFMuToDiDyYu9TQ
98,So nobody is going to talk about “HELA-HARD”💀,@Moistcomforter,UCRph8EN8tm7GhLnpVu6kG1Q


tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Device set to use cuda:0


    Unnamed: 0                         cid  \
88          88  UgyfQoYftAmSVxUnL294AaABAg   
89          89  Ugx7XgycIllbijorPfh4AaABAg   
90          90  UgxKdXjidGb-aem26sB4AaABAg   
91          91  UgyzBGmTZeiiz9LGUBN4AaABAg   
92          92  UgzPEWILHRnapaeI9Yp4AaABAg   
93          93  UgwIuafY1-tOiHwB4Al4AaABAg   
94          94  Ugy4hbIWoxLB58top6p4AaABAg   
95          95  UgyJiUDhU1Qm3Gmkux54AaABAg   
96          96  UgxWhkPCKi4-EWenQwp4AaABAg   
1            1  Ugxc-H1zW6zflmPjiTF4AaABAg   

                                                 text         time  \
88                Ngl invisible woman looks pretty OP    1 day ago   
89               Great video ( I watched 10 seconds )    1 day ago   
90                              0:00 that’s Fantastic    1 day ago   
91   She is somewhat broken compared to other healers  7 hours ago   
92  I despise Sombra so I hope Sue can hold her ow...    1 day ago   
93  completely unrelated to what comments may lay ...    1 day ago   
94 

    Unnamed: 0                         cid  \
8            8  UgzTls77_aFjRtc_ICJ4AaABAg   
9            9  UgzgjDjrQN0JMqy-GaZ4AaABAg   
10          10  UgydKL983RRfNvTf5JF4AaABAg   
11          11  UgyRzk2jdSu09qn6cxR4AaABAg   
12          12  UgzCUG16xsIgvCiVsnB4AaABAg   
13          13  UgxAzvM9UQhkzy9-kCx4AaABAg   
14          14  Ugx9Iwmy_lP6CCTNFrN4AaABAg   
15          15  Ugy9It0IGanVMhEaU8R4AaABAg   
97          97  UgwjDAbg_TjpnCrUQkd4AaABAg   
89          89  Ugx7XgycIllbijorPfh4AaABAg   

                                                 text                time  \
8   The fact that Hugh Jackman also played Van Hel...  1 day ago (edited)   
9   I’m imagine being unable to positions your shi...  1 day ago (edited)   
10  5:43\nActually the outfit this skin is based o...           1 day ago   
11  Invisible woman is really fun to sneak around ...        16 hours ago   
12  Less armour/clothing a Female characters has m...  1 day ago (edited)   
13  they could remove any two a